In [1]:
import sys; sys.path.append('../')
from antfarm.utils.sim import AntFarm
from antfarm.models.walks import Brownian, Vicsek

In [2]:
AntFarm(Brownian()).start()

Canvas(width=500)

In [3]:
AntFarm(Vicsek()).start()

Canvas(width=500)